In [17]:
import pandas as pd
import math
from datetime import datetime, timedelta
import torch
import joblib
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# ===============================
# 1. Definir la arquitectura del modelo
# ===============================
class DotacionPersonalNN(torch.nn.Module):
    def __init__(self):
        super(DotacionPersonalNN, self).__init__()
        self.layer1 = torch.nn.Linear(4, 64)
        self.layer2 = torch.nn.Linear(64, 32)
        self.layer3 = torch.nn.Linear(32, 1)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# ===============================
# 2. Cargar el modelo y el scaler
# ===============================
model_path = 'model.pth'
scaler_path = 'scaler.pkl'

model = DotacionPersonalNN()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

scaler = joblib.load(scaler_path)

# ===============================
# 3. Función para generar las predicciones masivas
# ===============================
def generar_predicciones_masivas(suc_ids, fecha_inicio, fecha_fin, model, scaler, db_connection_string):
    try:
        # Crear conexión a la base de datos (para lectura)
        engine = create_engine(db_connection_string)

        # Generar rango de fechas en formato 'YYYY-MM-DD'
        fechas_a_validar = pd.date_range(fecha_inicio, fecha_fin).strftime('%Y-%m-%d').tolist()

        # Consultar las transacciones por sucursal y fecha
        transacciones_query = """
        SELECT suc_id, fecha_id, hora_id, MAX(conteo_transacciones) AS max_transacciones
        FROM BI_Kielsa_Hecho_ProyeccionVenta_SucCanHora
        WHERE suc_id IN ({}) AND fecha_id BETWEEN '{}' AND '{}'
        GROUP BY suc_id, fecha_id, hora_id
        """.format(",".join(map(str, suc_ids)), fecha_inicio, fecha_fin)
        transacciones = pd.read_sql(transacciones_query, engine)
        
        # Convertir la columna 'fecha_id' a string con el formato 'YYYY-MM-DD'
        transacciones['fecha_id'] = pd.to_datetime(transacciones['fecha_id']).dt.strftime('%Y-%m-%d')
        
        # Lista para almacenar los resultados
        resultados = []

        for suc_id in suc_ids:
            for fecha_validada in fechas_a_validar:
                # Calcular el día de la semana (1=Lunes, 7=Domingo)
                dia_semana = datetime.strptime(fecha_validada, "%Y-%m-%d").weekday() + 1

                # Filtrar las transacciones de la sucursal y fecha actual
                transacciones_dia = transacciones[
                    (transacciones['suc_id'] == suc_id) &
                    (transacciones['fecha_id'] == fecha_validada)
                ]
                
                # Para cada registro de transacción, se genera la predicción
                for _, transaccion in transacciones_dia.iterrows():
                    hora = transaccion['hora_id']
                    transacciones_totales = math.ceil(transaccion['max_transacciones'])

                    # Preparar los datos de entrada para el modelo
                    input_data = pd.DataFrame({
                        'Suc_Id': [suc_id],
                        'Dia_Semana': [dia_semana],
                        'Hora': [hora],
                        'Transacciones_Totales': [transacciones_totales]
                    })
                    
                    input_scaled = scaler.transform(input_data.values)
                    input_tensor = torch.tensor(input_scaled, dtype=torch.float32).view(1, -1)

                    # Predecir el personal necesario
                    with torch.no_grad():
                        personal_necesario = model(input_tensor).item()
                        # Se aplica un redondeo: si la predicción es mayor o igual a 1.2 se redondea hacia arriba,
                        # de lo contrario hacia abajo.
                        personal_necesario = math.ceil(personal_necesario) if personal_necesario >= 1.2 else math.floor(personal_necesario)

                    # Almacenar el resultado (aseguramos que al menos sea 1)
                    resultados.append({
                        "Fecha_Id": fecha_validada,
                        "Hora": hora,
                        "Dia_Semana": dia_semana,
                        "Cantidad_Transacciones": transacciones_totales,
                        "Personal_Necesario": max(1, personal_necesario),
                        "Suc_Id": suc_id
                    })

        # Convertir la lista de resultados a DataFrame
        resultados_df = pd.DataFrame(resultados)
        return resultados_df

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# ===============================
# 4. Extraer los IDs de todas las sucursales (dinámicamente)
# ===============================
# Connection string para lectura (BI_FARINTER)
read_connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/BI_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)

# Crear conexión para extraer los suc_ids
engine_read = create_engine(read_connection_string)
branch_ids_query = "SELECT DISTINCT suc_id FROM BI_Kielsa_Hecho_ProyeccionVenta_SucCanHora"
branch_ids_df = pd.read_sql(branch_ids_query, engine_read)
suc_ids = branch_ids_df["suc_id"].tolist()

print("Sucursales encontradas:", suc_ids)

# ===============================
# 5. Definir el rango de fechas (febrero y marzo)
# ===============================
fecha_inicio = "2025-02-14"
fecha_fin   = "2025-03-31"

# ===============================
# 6. Generar las predicciones masivas para todas las sucursales
# ===============================
resultados_df = generar_predicciones_masivas(suc_ids, fecha_inicio, fecha_fin, model, scaler, read_connection_string)

if resultados_df is not None:
    # ordenar por sucursal, dia, hora de menor a mayor
    resultados_df = resultados_df.sort_values(by=['Suc_Id', 'Fecha_Id', 'Hora'])
    print(resultados_df.head())
    # save the results
    resultados_df.to_csv('resultados.csv', index=False)
else:
    print("No se generaron resultados.")




/var/folders/5s/y64xy8cd6cd355kr26pngsrh0000gn/T/ipykernel_65027/686394886.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, 

Sucursales encontradas: [43, 18, 143, 68, 315, 138, 33, 93, 211, 154, 118, 153, 86, 186, 204, 221, 247, 187, 255, 36, 19, 261, 147, 276, 53, 241, 73, 83, 172, 254, 281, 113, 291, 100, 66, 274, 13, 38, 301, 72, 183, 133, 251, 200, 140, 65, 166, 106, 269, 81, 115, 132, 201, 96, 15, 32, 206, 79, 126, 253, 45, 146, 181, 240, 151, 213, 215, 304, 300, 287, 176, 47, 24, 44, 76, 119, 179, 145, 51, 279, 169, 144, 205, 101, 212, 111, 112, 273, 1, 11, 41, 26, 31, 266, 58, 130, 244, 209, 219, 192, 105, 249, 162, 313, 239, 3, 39, 4, 23, 30, 71, 184, 252, 191, 124, 159, 258, 259, 326, 327, 90, 139, 64, 309, 91, 98, 54, 158, 37, 5, 171, 262, 312, 245, 141, 94, 302, 109, 104, 173, 84, 307, 134, 198, 34, 9, 290, 170, 305, 238, 77, 69, 50, 102, 59, 127, 222, 137, 16, 197, 177, 202, 63, 207, 325, 150, 256, 117, 120, 224, 271, 152, 70, 103, 167, 48, 88, 29, 12, 217, 267, 97, 199, 320, 56, 190, 165, 22, 122, 216, 149, 42, 27, 2, 92, 82, 129, 317, 324, 250, 67, 220, 270, 99, 52, 163, 203, 265, 116, 49, 131,

In [19]:
write_connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/IA_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)

engine_write = create_engine(write_connection_string)

# Insertar los resultados en la tabla. Si la tabla ya existe, se agrega (append).
resultados_df.to_sql('IA_Farinter_PrediccionPersonal', engine_write, if_exists='append', index=False)

print("Los datos se han insertado correctamente en IA_Farinter_PrediccionPersonal.")


Los datos se han insertado correctamente en IA_Farinter_PrediccionPersonal.
